In [ ]:
import glob
import math
import uproot
import numpy as np
from numpy.lib.function_base import meshgrid
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["svg"]

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#just a fix for a bug
import plotly.io as pio
pio.renderers.default = 'iframe_connected'

plt.style.use(hep.style.ROOT)  # For now ROOT defaults to CMS

path1 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=3.0eV'
path2 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=4.0eV'
path3 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=6.1eV'
path4 = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=8.2eV'
path = r'../PYTHON_graphs/DATA/Theory/TFMox-S/enant=S_hel=+1_KE=11.5eV'

# glob has to be sorted (both in unix and windows)
all_files = sorted(glob.glob(path + "/**.dat"))

li = []
cosphi_th = []
test_opening = []
allnames=[]
colnames = ["phi", "cos(theta)", "value"]

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/,
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None)  # r"\s+" is a regex (regular expression)
    # or delim_whitespace=True, it is faste
    temp_name=(filename.replace(" ","")).split("/")[-1].split(".")[0] # adding a column to df with the file name with no spaces and extension
    cosphi_th.append((int(temp_name.split("_")[0]), int(temp_name.split("_")[1])))
    df["filename"]=temp_name
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th = np.array(cosphi_th)
frame = pd.concat(li, axis=0)
#this commmand avoids 1_1 followed by 1_10
frame_srt = frame.groupby("filename",sort=False)

for key, item in frame_srt:
    allnames.append(key)

frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
#look to the values in b1 map and found the MFPAD
#accordinf to automatic_72_CPR.f90 from philipp [1,6] is from [cos(0), cos(pi)] = [1,-1]
cosphi_adj=list(fugi.remap(cosphi_th,0.835,-0.835,-165,165))
# cosphi_adj=list(remap(cosphi_th,-1.,1.,-180,180))
cosphi_adj=np.around(cosphi_adj,3)

dfind=pd.DataFrame(cosphi_adj, columns=["ctheta","phi"])
dfind["item"]=allnames
dfind.loc[dfind["phi"] == -105.]

In [ ]:
#selecting the rght file according to the needs ["filename"] == "x_x"]
nitem="1_3"
frame.loc[frame["filename"] == nitem]  # first way of selecting the right file
frame_set = frame.set_index("filename")  # build a mul

# converting to numpy arrays to plot: here full, in 72 MFPADs .unique()tiindex using the categories of filename
phi = frame.loc[frame["filename"] == nitem].iloc[:, 0].to_numpy()  # phi
ctheta = frame.loc[frame["filename"] == nitem].iloc[:, 1].to_numpy()  # cos(theta)
counts = frame.loc[frame["filename"] == nitem].iloc[:, 2].to_numpy()  # counts

In [ ]:
fig,ax=plt.subplots()
titel_fig="MFPAD el "+str(nitem)
ax.contourf(np.unique(phi),np.unique(ctheta),counts.reshape(200,100).T, cmap="viridis")
# plt.imshow(mPECD_cos_sum, vmin=-0.3, vmax=0.3, extent=[phiM[0],phiM[-1],cosM[0],cosM[-1]], interpolation='none', cmap=cmap_temp, aspect="auto")
ax.set_title(titel_fig)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
fig.show()
fig.savefig('OUTPUTS/MFPAD_th_el_1_3.png')

In [ ]:
#convertion to cartesian coordinate in MF
theta_rad = np.arccos(ctheta)
# theta_rad = np.arccos(ctheta_fixed)
phi_rad = phi * np.pi/180.
# phi_rad = phi_fixed * np.pi/180.
countsn = counts
# countsn = counts_fixed

# convertion to spherical coordinates 1D vectors of shape (20000,1)
x = countsn * np.sin(theta_rad) * np.cos(phi_rad)
X = x.reshape(200,100)
y = countsn * np.sin(theta_rad) * np.sin(phi_rad)
Y = y.reshape(200,100)
z = countsn * np.cos(theta_rad)
Z = z.reshape(200,100)

d = np.sqrt(x**2+y**2+z**2)
d_matrix = np.sqrt(X**2+Y**2+Z**2)

fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

# ax.plot(counts_fixed[0:200]-counts[0:200],c="b",label='diff_counts[0:200]')
ax.plot(counts_fixed-counts,c="b",label='diff_counts')
ax.plot(a1-b1,c="g",label='diff_+phi_-phi')
ax.plot(a2-b2,c="y",label='diff_+ctheta_-ctheta')
# ax.plot(avgcos[0:200]-counts[0:200],c="r",label='diff_+avgcos_-counts[0:200]')
# ax.plot(avgphi[0:200]-counts[0:200],c="m",label='diff_+avgphi_-counts[0:200]')
# ax.set_yscale('log')
plt.legend(loc="best")
print(np.min(phi_fixed),np.max(phi_fixed),np.min(ctheta_fixed),np.max(ctheta_fixed))

In [ ]:
fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z,
                      opacity=0.5,
                      color="rgba(244,22,100,06)"
                 
 )])
fig.update_layout(title="TFMeOx MFPADs theory triangulated",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=ctheta, y=phi, z=counts,
                    mode='markers',
                    # mode='lines',
                    marker=dict(
                    size=1.5,
                    colorscale='Viridis',   # choose a colorscale
                    opacity=0.85,
                    showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs spherical coordinates",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# Go scatters do not provide a legend
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=d,            # set color to an array/list of desired values
                        colorscale='Viridis',   # choose a colorscale
                        opacity=1.,
                        showscale=True          # to show the legend according to the color
                   )
                )])

fig.update_layout(title="TFMeOx MFPADs theory scattered",
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                #   margin=dict(l=0, r=0, b=0, t=0))
fig.show()

the Z distribution has the ending and the starting data that aren't matching. This is regardless the reshaping

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, colorscale='Viridis', connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory sur NO FIXf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

It is possible to force the equality and therefore bridging the gap

In [ ]:
#fixing the stripe
Y[-1,0:100]=Y[0,0:100]
X[-1,0:100]=X[0,0:100]
Z[-1,0:100]=Z[0,0:100]

#to share these variables to other notebooks
%store X
%store Y
%store Z
%store d_matrix

In [ ]:
#fixed stripe
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], x=X[0:200,0:100], y=Y[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                # width=500,
                # height=500,
                # margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z[0:200,0:100], surfacecolor=d_matrix, connectgaps=True)])
fig.update_layout(title='TFMeOx MFPADs theory surf', autosize=False,
                #   width=500, height=500,
                #   margin=dict(l=65, r=50, b=65, t=90))
                  margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
plotly.__version__